In [43]:
import numpy as np
import os
import argparse
import random
from tqdm import tqdm

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
from martins.complex_transformer import ComplexTransformer
from dataset import TimeSeriesDataset

In [4]:
pwd

'/Users/tianqinli/Code/Working-on/time-series-domain-adaptation/JDA'

In [37]:
# fake args
parameters = {
    'data_path': '/Users/tianqinli/Code/Working-on/Russ/time-series-domain-adaptation/data_unzip/',
    'file': 'processed_file_{}.pkl',
    'task': '3Av2',
    'batch_size': 32,
    'lr_clf': 1e-3,
    'epochs': 2,
    'PATH': '/Users/tianqinli/Code/Working-on/Russ/time-series-domain-adaptation/JDA/data_results/models/'
}

class ARGS:
    def __init__(self, parameters):
        self.data_path = parameters['data_path']
        self.file = parameters['file']
        self.task = parameters['task']
        self.batch_size = parameters['batch_size']
        self.lr_clf = parameters['lr_clf']
        self.epochs = parameters['epochs']
        self.path = parameters['PATH']

        
args = ARGS(parameters)
# other parameters
seq_len = 10 
feature_dim = 160
d_out = 50 if args.task == "3Av2" else 65
device = torch.device("cuda:0")

In [7]:
class FNN(nn.Module):
    def __init__(self, d_in, d_h, d_out, dp):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(d_in, d_h)
        self.fc2 = nn.Linear(d_h, d_out)
        self.dp = nn.Dropout(dp)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp(x)
        x = self.fc2(x)

        return x
    

In [8]:
# Data loader
from dataset import TimeSeriesDataset


training_set = TimeSeriesDataset(root_dir=args.data_path, file_name=args.file.format(args.task), train=True)
test_set = TimeSeriesDataset(root_dir=args.data_path, file_name=args.file.format(args.task), train=False)






train_loader = DataLoader(training_set, batch_size=args.batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=args.batch_size, shuffle=True)

In [48]:
training_set

In [34]:
# test for data loader
tmp = 0
for batch_id, (x, y) in enumerate(train_loader):
#     print(batch_id)
#     print(x.shape)
#     print(y.shape)
    tmp += x.shape[0]
    
    


In [35]:
tmp

5000

In [14]:
# make complex transformer

encoder = ComplexTransformer(layers=1, 
                               time_step=seq_len, 
                               input_dim=feature_dim, 
                               hidden_size=512, 
                               output_dim=512, 
                               num_heads=8,
                               out_dropout=0.5)

if torch.cuda.is_available(): encoder.to(device)

In [41]:
CNet = FNN(d_in=feature_dim * 2 * seq_len, d_h=500, d_out=d_out, dp=0.5)
if torch.cuda.is_available(): 
    CNet = CNet.to(device)

params = list(encoder.parameters()) + list(CNet.parameters())
criterion = nn.CrossEntropyLoss()

if torch.cuda.is_available(): 
    criterion = criterion.cuda()

optimizer = torch.optim.Adam(params, lr=args.lr_clf)
scheduler = ReduceLROnPlateau(optimizer, 'min')
    
    
# Encoding by complex transformer
x_mean_tr = training_set.data_mean
x_std_tr = training_set.data_std 

x_mean_te = test_set.data_mean
x_std_te = test_set.data_std

best_acc_train = best_acc_test = 0

if not os.path.isdir(args.path):
    os.mkdir(args.path)
    
#### train
for epoch in range(args.epochs):
    """Training"""
    correct_train = 0
    total_bs_train = 0 # total batch size
    train_loss = 0
    for batch_id, (x, y) in enumerate(tqdm(train_loader)):
        if torch.cuda.is_available():
            x, y = x.to(device), y.to(device) 
        batch_size = x.shape[0]
        #normalize data
        x = (x - x_mean_tr) / x_std_tr
        # take the real and imaginary part out
        real = x[:,:,0].reshape(batch_size, seq_len, feature_dim).float()
        imag = x[:,:,1].reshape(batch_size, seq_len, feature_dim).float()
        if torch.cuda.is_available(): 
            real.to(device)
            imag.to(device)
        real, imag = encoder(real, imag)
        pred = CNet(torch.cat((real, imag), -1).reshape(x.shape[0], -1)) 
        loss = criterion(pred, y.argmax(-1))
        #print(pred.argmax(-1), y.argmax(-1))
        correct_train += (pred.argmax(-1) == y.argmax(-1)).sum().item()
        total_bs_train += y.shape[0]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.shape[0]
    train_acc = float(correct_train) / total_bs_train
    train_loss = train_loss / total_bs_train
    best_acc_train = max(best_acc_train, train_acc)
    
    PATH = args.path + "model.ep" + str(epoch)
    torch.save(CNet.state_dict(), PATH)
    print("train_acc: "+ str(train_acc) + "; train_loss: " + str(train_loss))

  0%|          | 0/157 [00:00<?, ?it/s]

train_acc: 0.151; train_loss: 3.465614217376709


100%|██████████| 157/157 [00:30<00:00,  5.08it/s]

train_acc: 0.232; train_loss: 2.867535138320923


In [33]:
# test saved 

0.0536

In [44]:
os.path.isdir(args.path)

True